In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator
import spacy
from pythainlp.tokenize import word_tokenize
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

# ✅ Task 1: Data Preprocessing
# ✅ Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Tokenization
nlp_en = spacy.load("en_core_web_sm")

def tokenize_en(text):
    return [token.text for token in nlp_en(text.lower().strip())]

def tokenize_th(text):
    return word_tokenize(text.lower().strip())

# ✅ Load dataset
def load_dataset(folder_path):
    dataset = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(folder_path, file)).dropna(subset=["en_text", "th_text"])
            dataset.extend(zip(df["en_text"], df["th_text"]))
    return dataset

dataset_folder = "scb-mt-en-th-2020_forTest"
dataset = load_dataset(dataset_folder)

# ✅ Split dataset into training and validation
train_data, valid_data = train_test_split(dataset, test_size=0.1, random_state=42)

# ✅ Build vocabulary
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

def yield_tokens(data, language):
    for src, trg in data:
        yield tokenize_en(src) if language == "en" else tokenize_th(trg)

vocab_en = build_vocab_from_iterator(yield_tokens(train_data, "en"), min_freq=2, specials=special_symbols, special_first=True)
vocab_th = build_vocab_from_iterator(yield_tokens(train_data, "th"), min_freq=2, specials=special_symbols, special_first=True)

vocab_en.set_default_index(vocab_en['<unk>'])
vocab_th.set_default_index(vocab_th['<unk>'])

# ✅ Text to tensor transformation (Moved before DataLoader)
def tensor_transform(tokens, vocab):
    return torch.tensor([vocab['<sos>']] + [vocab[token] for token in tokens] + [vocab['<eos>']], dtype=torch.long)

def text_pipeline_en(text):
    return tensor_transform(tokenize_en(text), vocab_en)

def text_pipeline_th(text):
    return tensor_transform(tokenize_th(text), vocab_th)

# ✅ DataLoader (Corrected and moved before training loop)
def collate_batch(batch):
    src_batch, src_len_batch, trg_batch = [], [], []
    for src, trg in batch:
        src_tokens = text_pipeline_en(src)
        trg_tokens = text_pipeline_th(trg)
        src_batch.append(src_tokens)
        trg_batch.append(trg_tokens)
        src_len_batch.append(len(src_tokens))

    src_batch = pad_sequence(src_batch, padding_value=vocab_en['<pad>'], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab_th['<pad>'], batch_first=True)

    return src_batch.to(device), torch.tensor(src_len_batch, dtype=torch.int64).to(device), trg_batch.to(device)

batch_size = 64  # Or your desired batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)

# ✅ Performance Tracking
performance_metrics = []

def safe_exp(value):
    return torch.exp(torch.tensor(value)).item() if value < 10 else float('inf')

# ✅ Task 2: Implement Attention Mechanisms
# ✅ Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(2 * hid_dim, hid_dim)  # Linear layer for encoder outputs

    def forward(self, src, src_len):
        embedded = self.dropout(self.embedding(src))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.cpu(), batch_first=True, enforce_sorted=False)
        packed_outputs, (hidden, cell) = self.rnn(packed_embedded)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)

        hidden = (hidden[0] + hidden[1]) / 2
        cell = (cell[0] + cell[1]) / 2

        outputs = self.linear(outputs)  # Apply linear transformation to encoder outputs

        return outputs, hidden, cell

# ✅ Decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.attention = attention
        self.rnn = nn.LSTM(emb_dim + enc_hid_dim, dec_hid_dim, batch_first=True)
        self.fc_out = nn.Linear(enc_hid_dim + dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))

        attn_weights = self.attention(hidden, encoder_outputs).unsqueeze(1)
        context = torch.bmm(attn_weights, encoder_outputs)

        rnn_input = torch.cat((embedded, context), dim=2)
        output, (hidden, cell) = self.rnn(rnn_input, (hidden.unsqueeze(0), cell.unsqueeze(0)))

        prediction = self.fc_out(torch.cat((output.squeeze(1), context.squeeze(1)), dim=1))
        return prediction, hidden.squeeze(0), cell.squeeze(0)

# ✅ Seq2Seq Model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_len, trg):
        encoder_outputs, hidden, cell = self.encoder(src, src_len)
        input_ = trg[:, 0]
        outputs = []
        for t in range(1, trg.shape[1]):
            output, hidden, cell = self.decoder(input_, hidden, cell, encoder_outputs)
            outputs.append(output)
            input_ = output.argmax(1)
        return torch.stack(outputs, dim=1)
# ✅ General Attention (Dot Product)
class GeneralAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):  # Add __init__
        super().__init__()
        # You can add a linear layer here if needed for dimension matching in the future
        # self.W = nn.Linear(dec_hid_dim, enc_hid_dim)  # Example if dimensions need adjustment

    def forward(self, s, h):
        # If you added a linear layer in __init__, use it here:
        # s = self.W(s)
        attn_scores = torch.bmm(h, s.unsqueeze(2)).squeeze(2)
        return torch.softmax(attn_scores, dim=1)

# ✅ Multiplicative Attention (Dot Product with Weight)
class MultiplicativeAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):  # Add __init__
        super().__init__()
        self.W = nn.Linear(dec_hid_dim, enc_hid_dim, bias=False)  # Corrected: dec_hid_dim -> enc_hid_dim

    def forward(self, s, h):
        s = self.W(s).unsqueeze(2)
        attn_scores = torch.bmm(h, s).squeeze(2)
        return torch.softmax(attn_scores, dim=1)

# ✅ Additive Attention (Bahdanau)
class AdditiveAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):  # Add __init__
        super().__init__()
        self.W1 = nn.Linear(enc_hid_dim, dec_hid_dim)
        self.W2 = nn.Linear(dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, s, h):
        s_expanded = self.W2(s).unsqueeze(1)
        energy = torch.tanh(self.W1(h) + s_expanded)
        attn_scores = self.v(energy).squeeze(2)
        return torch.softmax(attn_scores, dim=1)


# ✅ Task 3: Training and Evaluation for All Attention Types
attention_types = {"general": GeneralAttention, "multiplicative": MultiplicativeAttention, "additive": AdditiveAttention}
results = {}

def plot_attention(attn_weights, input_tokens, output_tokens):
    fig, ax = plt.subplots(figsize=(10, 8))
    cax = ax.matshow(attn_weights.cpu().detach().numpy(), cmap='viridis')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + input_tokens, rotation=90)
    ax.set_yticklabels([''] + output_tokens)
    plt.xlabel("Input Sequence")
    plt.ylabel("Output Sequence")
    plt.show()

for attn_name, attn_class in attention_types.items():
    print(f"\nTraining with {attn_name} attention...")
    
    encoder = Encoder(len(vocab_en), 256, 512, 0.5).to(device)
    attention = attn_class(512, 512).to(device)
    decoder = Decoder(len(vocab_th), 256, 512, 512, 0.5, attention).to(device)
    model = Seq2Seq(encoder, decoder).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss(ignore_index=vocab_th['<pad>'])

    training_losses, training_ppls, validation_losses, validation_ppls = [], [], [], []
    start_time = time.time()

    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for src, src_len, trg in dataloader:
            optimizer.zero_grad()
            output = model(src, src_len, trg)
            trg = trg[:, 1:].reshape(-1)
            output = output.reshape(-1, output.shape[-1])
            mask = trg != vocab_th['<pad>']
            output, trg = output[mask], trg[mask]
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(dataloader)
        training_losses.append(train_loss)
        training_ppls.append(safe_exp(train_loss))
        
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for src, src_len, trg in dataloader:
                output = model(src, src_len, trg)
                trg = trg[:, 1:].reshape(-1)
                output = output.reshape(-1, output.shape[-1])
                mask = trg != vocab_th['<pad>']
                output, trg = output[mask], trg[mask]
                loss = criterion(output, trg)
                valid_loss += loss.item()
        valid_loss /= len(dataloader)
        validation_losses.append(valid_loss)
        validation_ppls.append(safe_exp(valid_loss))
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train PPL: {training_ppls[-1]:.4f}, Validation Loss: {valid_loss:.4f}, Validation PPL: {validation_ppls[-1]:.4f}")
    
    end_time = time.time()
    training_time = end_time - start_time
    performance_metrics.append([attn_name, training_time])
    
    results[attn_name] = {
        "Training Loss": training_losses,
        "Training PPL": training_ppls,
        "Validation Loss": validation_losses,
        "Validation PPL": validation_ppls,
    }

def plot_losses(results):
    plt.figure(figsize=(10, 5))
    for attn_type, metrics in results.items():
        plt.plot(metrics["Training Loss"], label=f"{attn_type} - Train Loss")
        plt.plot(metrics["Validation Loss"], linestyle="dashed", label=f"{attn_type} - Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.show()

plot_losses(results)

# ✅ Print Performance Metrics
print("\nPerformance Metrics (Training Time per Attention Type):")
for attn_type, time_taken in performance_metrics:
    print(f"{attn_type}: {time_taken:.2f} seconds")

print("✅ Training Completed for All Attention Types")


KeyboardInterrupt: 